<a href="https://colab.research.google.com/github/thomas-greig/MSc/blob/main/compare_f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from numpy.fft import fft2, ifft2, fftshift
from numba import njit

# =========================
# Global Parameters (unchanged)
# =========================
Lx, Ly = 100, 100
N = 5000
steps = 300_000_000           # very heavy; reduce for speed if needed
b_over_T = 3                  # β/T
J_over_T = b_over_T/3.0       # J/T
eps = 0
sample_every = N*1000
relaxation_index = int(0.2 * steps / sample_every)

runs = 15
base_seed = 52341

# --- τ-leap parameters ---
D0 = 1.0
dt = 0.99 / (8.0 * D0)

# =========================
# Helpers (trimmed to essentials)
# =========================
def compute_2D_correlation(data):
    """Streaming FFT accumulation (float32/complex64)."""
    T, Lx_, Ly_ = data.shape
    mean_n = data.mean(axis=0).astype(np.float32, copy=False)
    corr_fft = np.zeros((Lx_, Ly_), dtype=np.complex64)
    for k in range(T):
        frame32 = data[k].astype(np.float32, copy=False)
        f = np.fft.fft2(frame32 - mean_n)
        corr_fft += f * np.conj(f)
    corr = np.real(np.fft.ifft2(corr_fft)).astype(np.float32, copy=False) / (T * Lx_ * Ly_)
    return fftshift(corr)

def directional_cuts(corr_2d):
    """Return r_x, cut_x, r_y, cut_y (cuts from centre toward +x and +y)."""
    Lx_, Ly_ = corr_2d.shape
    cx, cy = Lx_ // 2, Ly_ // 2
    cut_x = corr_2d[cx:, cy]
    cut_y = corr_2d[cx, cy:]
    r_x = np.arange(0, len(cut_x))
    r_y = np.arange(0, len(cut_y))
    return r_x, cut_x, r_y, cut_y

def powerlaw_fit(r, c, r_window=None, use_abs=False, positive_only=False, min_pts=2):
    """Fit log(y) vs log(r) via linear regression."""
    y = np.abs(c) if use_abs else c
    mask = np.isfinite(r) & np.isfinite(y) & (y > 0)
    if r_window is not None:
        rmin, rmax = r_window
        mask &= (r > rmin) & (r < rmax)
    if np.count_nonzero(mask) < min_pts:
        return np.nan, np.nan, np.nan, mask
    lr = np.log(r[mask]); ly = np.log(y[mask])
    slope, intercept, r_value, _, _ = linregress(lr, ly)
    return slope, intercept, r_value**2, mask

def stats_arr(arr):
    a = arr[np.isfinite(arr)]
    if a.size == 0: return np.nan, np.nan
    return np.mean(a), (np.std(a, ddof=1) if a.size > 1 else 0.0)

# =========================
# τ-LEAP SIMULATION (Numba) — pass DRIFT_ALIGN explicitly
# =========================
@njit
def nn_sum_numba(occ, x, y, Lx, Ly):
    return (occ[(x-1)%Lx,y] + occ[(x+1)%Lx,y] +
            occ[x,(y-1)%Ly] + occ[x,(y+1)%Ly])

@njit
def one_sweep_tau(positions, occupancy, Lx, Ly, b_over_T, J_over_T, DRIFT_ALIGN):
    """
    τ-leap (discrete-time CTMC):
      ΔE_tot = ΔE_on + ΔE_nn - drive,  where drive encodes drift via DRIFT_ALIGN.
      λ_dir = 2*D0 * sigmoid(ΔE_tot),
      p_dir = λ_dir * dt,  p_stay = 1 - Σ p_dir.
    """
    N = positions.shape[0]

    # Random permutation (without replacement) per sweep
    order = np.arange(N)
    for i in range(N-1, 0, -1):
        j = np.random.randint(0, i+1)
        order[i], order[j] = order[j], order[i]

    for t in range(N):
        idx = order[t]
        x0 = positions[idx,0]; y0 = positions[idx,1]

        n0 = occupancy[x0,y0]
        S0 = (occupancy[(x0-1)%Lx,y0] + occupancy[(x0+1)%Lx,y0] +
              occupancy[x0,(y0-1)%Ly] + occupancy[x0,(y0+1)%Ly])

        # RIGHT
        xr, yr = (x0+1)%Lx, y0
        nr = occupancy[xr,yr]
        Sr = (occupancy[(xr-1)%Lx,yr] + occupancy[(xr+1)%Lx,yr] +
              occupancy[xr,(yr-1)%Ly] + occupancy[xr,(yr+1)%Ly])
        dE_on = 2.0*b_over_T*(1.0 + (nr - n0))
        dE_nn = - J_over_T * ((Sr - S0) - 1.0)
        drive = float(DRIFT_ALIGN[1])   # Δx = +1 for RIGHT
        dE = dE_on + dE_nn - drive
        br = 1.0/(1.0 + np.exp(dE))
        lam_r = 2.0*D0 * br

        # LEFT
        xl, yl = (x0-1)%Lx, y0
        nl = occupancy[xl,yl]
        Sl = (occupancy[(xl-1)%Lx,yl] + occupancy[(xl+1)%Lx,yl] +
              occupancy[xl,(yl-1)%Ly] + occupancy[xl,(yl+1)%Ly])
        dE_on = 2.0*b_over_T*(1.0 + (nl - n0))
        dE_nn = - J_over_T * ((Sl - S0) - 1.0)
        drive = float(DRIFT_ALIGN[0])   # Δx = -1 for LEFT
        dE = dE_on + dE_nn - drive
        bl = 1.0/(1.0 + np.exp(dE))
        lam_l = 2.0*D0 * bl

        # UP
        xu, yu = x0, (y0+1)%Ly
        nu = occupancy[xu,yu]
        Su = (occupancy[(xu-1)%Lx,yu] + occupancy[(xu+1)%Lx,yu] +
              occupancy[xu,(yu-1)%Ly] + occupancy[xu,(yu+1)%Ly])
        dE_on = 2.0*b_over_T*(1.0 + (nu - n0))
        dE_nn = - J_over_T * ((Su - S0) - 1.0)
        drive = float(DRIFT_ALIGN[2])   # Δx = 0 for UP
        dE = dE_on + dE_nn - drive
        bu = 1.0/(1.0 + np.exp(dE))
        lam_u = 2.0*D0 * bu

        # DOWN
        xd, yd = x0, (y0-1)%Ly
        nd = occupancy[xd,yd]
        Sd = (occupancy[(xd-1)%Lx,yd] + occupancy[(xd+1)%Lx,yd] +
              occupancy[xd,(yd-1)%Ly] + occupancy[xd,(yd+1)%Ly])
        dE_on = 2.0*b_over_T*(1.0 + (nd - n0))
        dE_nn = - J_over_T * ((Sd - S0) - 1.0)
        drive = float(DRIFT_ALIGN[3])   # Δx = 0 for DOWN
        dE = dE_on + dE_nn - drive
        bd = 1.0/(1.0 + np.exp(dE))
        lam_d = 2.0*D0 * bd

        # probabilities p = λ Δt; remainder is stay
        pr = lam_r * dt
        pl = lam_l * dt
        pu = lam_u * dt
        pd = lam_d * dt
        psum = pr + pl + pu + pd

        if psum > 1.0:
            s = 0.999999 / psum
            pr *= s; pl *= s; pu *= s; pd *= s
            psum = pr + pl + pu + pd

        u = np.random.random()
        if u < pr:
            x1, y1 = (x0+1)%Lx, y0
        elif u < pr + pl:
            x1, y1 = (x0-1)%Lx, y0
        elif u < pr + pl + pu:
            x1, y1 = x0, (y0+1)%Ly
        elif u < psum:
            x1, y1 = x0, (y0-1)%Ly
        else:
            x1, y1 = x0, y0  # stay

        if (x1 != x0) or (y1 != y0):
            positions[idx,0] = x1; positions[idx,1] = y1
            occupancy[x0,y0] -= 1; occupancy[x1,y1] += 1

@njit
def simulate_numba(Lx, Ly, N, steps, sample_every, b_over_T, J_over_T, seed, DRIFT_ALIGN):
    if seed >= 0:
        np.random.seed(seed)
    positions = np.empty((N,2), dtype=np.int32)
    positions[:,0] = np.random.randint(0, Lx, size=N)
    positions[:,1] = np.random.randint(0, Ly, size=N)
    occupancy = np.zeros((Lx, Ly), dtype=np.int16)
    for i in range(N):
        occupancy[positions[i,0], positions[i,1]] += 1
    n_samples = steps // sample_every + 1
    series = np.zeros((n_samples, Lx, Ly), dtype=np.int16)
    sidx = 0; series[sidx,:,:] = occupancy; sidx += 1
    step = 0
    while step < steps:
        one_sweep_tau(positions, occupancy, Lx, Ly, b_over_T, J_over_T, DRIFT_ALIGN)
        step += N
        while (sidx < n_samples) and (step >= sidx*sample_every):
            series[sidx,:,:] = occupancy
            sidx += 1
            if sidx >= n_samples:
                break
    return series

# =========================
# Per-run stats (no plotting)
# =========================
def compute_run_stats(series, window=(1,20)):
    stationary = series[relaxation_index:]
    corr_2d = compute_2D_correlation(stationary)
    r_x, cut_x, r_y, cut_y = directional_cuts(corr_2d)
    sx_all, _, r2x_all, _ = powerlaw_fit(r_x[1:], cut_x[1:], r_window=None, use_abs=False, positive_only=True)
    sy_all_mag, _, r2y_all, _ = powerlaw_fit(r_y[1:], cut_y[1:], r_window=None, use_abs=True,  positive_only=False)
    sx_rng, ix_rng, r2x_rng, maskx = powerlaw_fit(r_x[1:], cut_x[1:], r_window=window, use_abs=False, positive_only=True)
    sy_rng_mag, iy_rng, r2y_rng, masky = powerlaw_fit(r_y[1:], cut_y[1:], r_window=window, use_abs=True,  positive_only=False)
    return corr_2d, (sx_all, sy_all_mag, sx_rng, sy_rng_mag, r2x_all, r2y_all, r2x_rng, r2y_rng), ((r_x[1:], cut_x[1:], maskx, ix_rng, sx_rng),
                                                                                                   (r_y[1:], np.abs(cut_y[1:]), masky, iy_rng, sy_rng_mag))

# =========================
# Run for f in {1,2,3}, aggregate, and plot shared axes
# =========================
f_values = [1, 2, 3]
window = (1, 20)

summary = {}          # store mean±sd slopes per f
fit_lines_x = {}      # per f: (rr, yy) for x-cut
fit_lines_y = {}      # per f: (rr, yy) for y-cut

colors = {1: 'C0', 2: 'C1', 3: 'C2'}

for f in f_values:
    # Drift along +x with magnitude f: DRIFT_ALIGN = [-f, +f, 0, 0] for [LEFT, RIGHT, UP, DOWN]
    DRIFT_ALIGN = np.array([-f, +f, 0, 0], dtype=np.int8)

    # Accumulators
    sum_corr_2d = None
    sx_rng_list, sy_rng_list = [], []

    for i in range(runs):
        seed = base_seed + 1000*f + i
        series = simulate_numba(Lx, Ly, N, steps, sample_every, b_over_T, J_over_T, seed, DRIFT_ALIGN)

        corr_2d, stats, cutpacks = compute_run_stats(series, window=window)
        sx_all, sy_all, sx_rng, sy_rng, *_ = stats
        sx_rng_list.append(sx_rng)
        sy_rng_list.append(sy_rng)

        if sum_corr_2d is None:
            sum_corr_2d = np.zeros_like(corr_2d, dtype=np.float64)
        sum_corr_2d += corr_2d

    # Averages across runs
    avg_corr_2d = sum_corr_2d / runs
    mean_sx, sd_sx = stats_arr(np.array(sx_rng_list, dtype=float))
    mean_sy, sd_sy = stats_arr(np.array(sy_rng_list, dtype=float))
    summary[f] = dict(mean_sx=mean_sx, sd_sx=sd_sx, mean_sy=mean_sy, sd_sy=sd_sy)

    # Build fit lines using the averaged correlation and the same window
    r_x, cut_x, r_y, cut_y = directional_cuts(avg_corr_2d)
    rx, cx = r_x[1:], cut_x[1:]
    ry, cy_abs = r_y[1:], np.abs(cut_y[1:])

    sx_fit, ix_fit, _, maskx = powerlaw_fit(rx, cx, r_window=window, use_abs=False, positive_only=True)
    sy_fit, iy_fit, _, masky = powerlaw_fit(ry, cy_abs, r_window=window, use_abs=False, positive_only=False)

    # Smooth lines across the masked r-range
    if np.any(maskx):
        rr_x = np.linspace(rx[maskx].min(), rx[maskx].max(), 200)
        yy_x = np.exp(ix_fit) * rr_x**sx_fit
        fit_lines_x[f] = (rr_x, yy_x)
    else:
        fit_lines_x[f] = (np.array([1,2]), np.array([np.nan, np.nan]))

    if np.any(masky):
        rr_y = np.linspace(ry[masky].min(), ry[masky].max(), 200)
        yy_y = np.exp(iy_fit) * rr_y**sy_fit
        fit_lines_y[f] = (rr_y, yy_y)
    else:
        fit_lines_y[f] = (np.array([1,2]), np.array([np.nan, np.nan]))

# =========================
# Plot: X-cut fits (C>0) overlaid for all f
# =========================
plt.figure()
for f in f_values:
    rr, yy = fit_lines_x[f]
    m, s = summary[f]['mean_sx'], summary[f]['sd_sx']
    plt.loglog(rr, yy, '-', label=f"f={f}  slope={m:.3f} ± {s:.3f}", color=colors[f])
plt.xlabel('r (lattice units)')
plt.ylabel('C_x(r)')
plt.grid(True, which="both", ls="-", alpha=0.5)
plt.legend()
plt.show()

# =========================
# Plot: Y-cut fits (|C|) overlaid for all f
# =========================
plt.figure()
for f in f_values:
    rr, yy = fit_lines_y[f]
    m, s = summary[f]['mean_sy'], summary[f]['sd_sy']
    plt.loglog(rr, yy, '-', label=f"f={f}  slope={m:.3f} ± {s:.3f}", color=colors[f])
plt.xlabel('r (lattice units)')
plt.ylabel('|C_y(r)|')
plt.grid(True, which="both", ls="-", alpha=0.5)
plt.legend()
plt.show()

# =========================
# Console summary
# =========================
for f in f_values:
    print(f"f={f}:  x-cut slope (1<r<20) = {summary[f]['mean_sx']:.4f} ± {summary[f]['sd_sx']:.4f}   "
          f"|  y-cut slope (|C|, 1<r<20) = {summary[f]['mean_sy']:.4f} ± {summary[f]['sd_sy']:.4f}")
